## Package Import

In [0]:
import cv2
import os
import pandas as pd

## Setting Up GDrive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
%cd /content/drive/My Drive/Colab Notebooks/I&D_Whale/data/

/content/drive/My Drive/Colab Notebooks/I&D_Whale/data


In [0]:
# !unzip train.zip
# !unzip test.zip 

## Creating train.csv and test.csv

In [0]:
train = []
for path, subdirs, files in os.walk('train/train/'):
    for name in files:
        train.append(os.path.join(path, name))

test = []
for path, subdirs, files in os.walk('test/'):
    for name in files:
        test.append(os.path.join(path, name))

In [0]:
# # For Train
# df = pd.DataFrame([(os.listdir(f'train/train/{x}'), x) for x in os.listdir('train/train/')], 
#                   columns=['Image', 'Id'])
# lst_col = 'Image'

# df = pd.DataFrame({
#       col:np.repeat(df[col].values, df[lst_col].str.len())
#       for col in df.columns.drop(lst_col)}
#     ).assign(**{lst_col:np.concatenate(df[lst_col].values)})[df.columns]

# df['fil_path'] = train
# df.to_csv('train.csv', index=False)

# # For Test
# df = pd.DataFrame([(x, 0) for x in os.listdir('test/')], 
#                   columns=['Image', 'Id'])
# df['fil_path'] = test
# df.to_csv('test.csv', index=False)

In [97]:
df.head()

,Image,Id,fil_path
0,PM-WWA-20170321-046.jpg,0,test/PM-WWA-20170321-046.jpg
1,PM-WWA-20170321-053.jpg,0,test/PM-WWA-20170321-053.jpg
2,PM-WWA-20170321-322.jpg,0,test/PM-WWA-20170321-322.jpg
3,PM-WWA-20170321-332.jpg,0,test/PM-WWA-20170321-332.jpg
4,PM-WWA-20170321-344.jpg,0,test/PM-WWA-20170321-344.jpg


In [128]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
tr_list = [(x,p) for _,x,p in train[['Image', 'fil_path']].to_records()]
te_list = [(x,p) for _,x,p in test[['Image', 'fil_path']].to_records()]
join = dict(tr_list + te_list)
len(join), len(tr_list), len(te_list)

(5347, 4539, 808)

In [137]:
for p in join:
    print(p, join[p])
    break

PM-WWA-20050416-037.jpg train/train/-1/PM-WWA-20050416-037.jpg


In [141]:
# Determise the size of each image
from os.path import isfile
from PIL import Image as pil_image
from tqdm import tqdm_notebook

def expand_path(p):
    if isfile(join[p]):
        return join[p]
    
p2size = {}
for p in tqdm_notebook(join):
    size      = pil_image.open(expand_path(p)).size
    p2size[p] = size
len(p2size), list(p2size.items())[:5]

(5347,
 [('PM-WWA-20050416-037.jpg', (3072, 2048)),
  ('PM-WWA-20050513-229.jpg', (504, 356)),
  ('PM-WWA-20050513-243.jpg', (852, 496)),
  ('PM-WWA-20050520-134.jpg', (367, 181)),
  ('PM-WWA-20050612-047.jpg', (932, 584))])

In [139]:
!pip install imagehash

# A Heuristic Based Approach to identify Duplicate Images

In [0]:
# # Read or generate p2h, a dictionary of image name to image id (picture to hash)
# import pickle
# import numpy as np
# from imagehash import phash
# from math import sqrt


# # Two phash values are considered duplicate if, for all associated image pairs:
# # 1) They have the same mode and size;
# # 2) After normalizing the pixel to zero mean and variance 1.0, the mean square error does not exceed 0.1
# def match(h1,h2):
#     for p1 in h2ps[h1]:
#         for p2 in h2ps[h2]:
#             i1 =  pil_image.open(expand_path(p1))
#             i2 =  pil_image.open(expand_path(p2))
#             if i1.mode != i2.mode or i1.size != i2.size: return False, None
#             a1 = np.array(i1)
#             a1 = a1 - a1.mean()
#             a1 = a1/sqrt((a1**2).mean())
#             a2 = np.array(i2)
#             a2 = a2 - a2.mean()
#             a2 = a2/sqrt((a2**2).mean())
#             a  = ((a1 - a2)**2).mean()
#             # print(a)
#             if a > -1: return False, a
#     return True, a


# if isfile('pickled_obj/p1.pickle'):
#     with open('pickled_obj/p2h.pickle', 'rb') as f:
#         p2h = pickle.load(f)
# else:
#     # Compute phash for each image in the training and test set.
#     p2h = {}
#     for p in tqdm_notebook(join):
#         img    = pil_image.open(expand_path(p))
#         h      = phash(img)
#         p2h[p] = h

#     # Find all images associated with a given phash value.
#     h2ps = {}
#     for p,h in p2h.items():
#         if h not in h2ps: h2ps[h] = []
#         if p not in h2ps[h]: h2ps[h].append(p)

#     # Find all distinct phash values
#     hs = list(h2ps.keys())

#     # If the images are close enough, associate the two phash values (this is the slow part: n^2 algorithm)
#     h2h = {}
#     h1_h2 = []
#     a_val = []
#     for i,h1 in enumerate(tqdm_notebook(hs)):
#         for h2 in hs[:i]:
#             # print(match(h1, h2))
#             flag, a = match(h1, h2)
#             a_val.append(a)
#             h1_h2.append(h1-h2)
#             if (h1-h2 <= 6) and (flag=='True'):
#                 s1 = str(h1)
#                 s2 = str(h2)
#                 if s1 < s2: s1,s2 = s2,s1
#                 h2h[s1] = s2

#     # Group together images with equivalent phash, and replace by string format of phash (faster and more readable)
#     for p,h in p2h.items():
#         h = str(h)
#         if h in h2h: h = h2h[h]
#         p2h[p] = h

# len(p2h), list(p2h.items())[:5]

In [0]:
# os.mkdir('pickled_obj')      #Open this only if you don't have a folder called 'pickled_obj'
# with open('pickled_obj/p2h.pickle', 'wb') as fp:
#     pickle.dump(p2h, fp)

In [147]:
# For each image id, determine the list of pictures
h2ps = {}
for p,h in p2h.items():
    if h not in h2ps: h2ps[h] = []
    if p not in h2ps[h]: h2ps[h].append(p)
# Notice how 25460 images use only 20913 distinct image ids.
len(h2ps),list(h2ps.items())[:5]

(5347,
 [('c186fc81c3dca3e3', ['PM-WWA-20050416-037.jpg']),
  ('f69588d7660198ee', ['PM-WWA-20050513-229.jpg']),
  ('e3289cc3b73c2cc3', ['PM-WWA-20050513-243.jpg']),
  ('f9929c78c369a436', ['PM-WWA-20050520-134.jpg']),
  ('b3eccc23722c8c97', ['PM-WWA-20050612-047.jpg'])])

In [154]:
df = pd.DataFrame(h2ps.items(), columns=['hash', 'imgs'])
df['cc'] = df['imgs'].apply(lambda x: len(x))
df[df['cc']>1]

,hash,imgs,cc


In [0]:
# Show an example of a duplicate image (from training of test set)
import matplotlib.pyplot as plt

def show_whale(imgs, per_row=2):
    n         = len(imgs)
    rows      = (n + per_row - 1)//per_row
    cols      = min(per_row, n)
    fig, axes = plt.subplots(rows,cols, figsize=(24//per_row*cols,24//per_row*rows))
    for ax in axes.flatten(): ax.axis('off')
    for i,(img,ax) in enumerate(zip(imgs, axes.flatten())): ax.imshow(img.convert('RGB'))

for h, ps in h2ps.items():
    if len(ps) > 2:
        print('Images:', ps)
        imgs = [pil_image.open(expand_path(p)) for p in ps]
        show_whale(imgs, per_row=len(ps))
        break